In [ ]:
from collections import Counter
from nltk.corpus import stopwords
!pip install scikit-learn
!pip install rouge
!pip install nltk
from rouge import Rouge 
import nltk
import nltk.translate.bleu_score as bleu
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize, sent_tokenize 
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
import re
import nltk
from nltk.translate.bleu_score import sentence_bleu
nltk.download('stopwords')
import os,glob,re

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
rouge1_f1_5 = []
rouge1_f1_10 = []
rouge1_f1_15 = []
rouge1_f1_20 = []

bleu_5 = []
bleu_10 = []
bleu_15 = []
bleu_20 = []

sum_len_w = []
sum_len_l = []

In [ ]:
def extract_keywords(text, n_keywords):
    # Обозначаем текст
    tokens = text.lower().split()

    # Удаляем стоп-слова
    stop_words = set(stopwords.words('russian'))
    tokens = [token for token in tokens if token not in stop_words]

    # Вычислим частоту каждого слова
    freq = Counter(tokens)

    # Присваиваем баллы каждому слову в зависимости от частоты и положения
    scores = {word: freq[word] * (i+1) for i, word in enumerate(tokens)}

    #Отсортируем слова по количеству баллов и выбираем лучшие n_keywords
    keywords = sorted(scores.items(), key=lambda x: x[1], reverse=True)[:n_keywords]

    # Возвращаем лучшие ключевые слова
    return [keyword[0] for keyword in keywords]

def summary_to_sentences(summary):
# Разделм краткое содержание на предложения, используя символ '.' в качестве разделителя
    sentences = re.split("[.!?]",summary)
    
    #Преобразуем каждое предложение в список слов
    sentence_lists = [sentence.split() for sentence in sentences]
    
    return sentence_lists

def paragraph_to_wordlist(paragraph):
    # Разделим абзац на слова, используя пробел в качестве разделителя
    words = paragraph.split()
    return words


In [ ]:
path = "/content/drive/My Drive/texts/London J"
for filename in glob.glob(os.path.join(path, '*.txt')):
    with open(os.path.join(os.getcwd(), filename), 'r') as f: 
      text = f.read().replace('\n', '').replace('\r', '')
    i = 5
    n_keywords = 10
    while i < 21:

        keywords = extract_keywords(text, n_keywords)

        # Обобщим текст, используя основные ключевые слова
        sentences = re.split("[.!?]",text)
        k = 0
        summary = ''
        for sentence in sentences:
            for keyword in keywords:
                if keyword in sentence.lower():
                    summary += sentence.strip() + '. '
                    k = k+1
                    break
        if (k == i):
          break

        rouge = Rouge()
        scores = rouge.get_scores(summary, text)

        reference_paragraph = text
        reference_summary = summary_to_sentences(reference_paragraph)
        predicted_paragraph = summary
        predicted_summary = paragraph_to_wordlist(predicted_paragraph)

        score = sentence_bleu(reference_summary, predicted_summary)

        match i:
          case 5:
            rouge1_f1_5.append(scores[0]['rouge-1']['f'])
            bleu_5.append(score)
          case 10:
            rouge1_f1_10.append(scores[0]['rouge-1']['f'])
            bleu_10.append(score) 
          case 15:
            rouge1_f1_15.append(scores[0]['rouge-1']['f'])
            bleu_15.append(score)
          case 20:
            rouge1_f1_20.append(scores[0]['rouge-1']['f'])
            bleu_20.append(score)    
        sum_len_l.append(len(summary))
        sum_len_w.append(len(summary.split()))
        i = i + 5


In [ ]:
import csv

fields1 = [ 'sum_len_l','sum_len_w']
rows1 = [sum_len_l, sum_len_w]

fields2 = ['rouge1_f1_5','bleu5','rouge1_f1_10','bleu10','rouge1_f1_15','bleu_15','rouge1_f1_20','bleu_20']
rows2 = [rouge1_f1_5,bleu_5,rouge1_f1_10,bleu_10,rouge1_f1_15,bleu_15,rouge1_f1_20,bleu_20]

from itertools import zip_longest

export_data = zip_longest(*rows1, fillvalue = '')
with open('len_short_LuhnLondon.csv', 'w', encoding="utf-8", newline='') as myfile:
      wr = csv.writer(myfile)
      wr.writerow((fields1))
      wr.writerows(export_data)
myfile.close()


export_data = zip_longest(*rows2, fillvalue = '')
with open('metric_short_LuhnLondon.csv', 'w', encoding="utf-8", newline='') as myfile:
      wr = csv.writer(myfile)
      wr.writerow((fields2))
      wr.writerows(export_data)
myfile.close()